In [15]:
import numpy as np
import re
puzzle = (open("AoC22.txt", "r").read().splitlines())
bossHp = int(puzzle[0].split(" ")[2])
bossDmg = int(puzzle[1].split(" ")[1])

In [16]:
#spell - cost, dmg, heal, armor, mana, effect
spells = [[53, 4, 0, 0, 0, 0],[73, 2, 2, 0, 0, 0],[113, 0, 0, 7, 0, 6],[173, 3, 0, 0, 0, 6],[229, 0, 0, 0, 101, 5]]

In [31]:
meHp =  50
meMana = 500
meArmor = 0
meManaSpent = 0
from functools import lru_cache
@lru_cache
def simulatePlayer(mehp, memana, mearmor, memanaspent, bosshp, bossdmg, effects):
    #simulates a player turn
    #returns the minimum mana spent to win
    #apply effects
    if effects[0] > 0:
        bosshp -= 4
        effects[0] -= 1
    if effects[1] > 0:
        mearmor = 7
        effects[1] -= 1
    if effects[2] > 0:
        mehp += 101
        effects[2] -= 1
    if effects[3] > 0:
        effects[3] -= 1
    if effects[4] > 0:
        memana += 101
        effects[4] -= 1
    #check if boss is dead
    if bosshp <= 0:
        return (True, memanaspent)
    #check if player is dead
    if mehp <= 0:
        return (False, memanaspent)
    poss = []
    #Simulate player turn for every possible spell
    for i in range(len(spells)):
        if spells[i][0] <= memana and effects[i] == 0:
            #simulate spell
            newHp = mehp
            newMana = memana - spells[i][0]
            newArmor = mearmor
            newManaSpent = memanaspent + spells[i][0]
            newEffects = effects
            newEffects[i] = spells[i][5]
            newBossHp = bosshp - spells[i][1]
            newBossDmg = bossDmg - spells[i][3]
            if simulateBoss(newHp, newMana, newArmor, newManaSpent, newBossHp, newBossDmg, newEffects):
                poss.append(simulateBoss(newHp, newMana, newArmor, newManaSpent, newBossHp, newBossDmg, newEffects))
    if len(poss) == 0:
        return (False, memanaspent)
    return min(poss,key=lambda x: x[1])

from functools import lru_cache
@lru_cache
def simulateBoss(mehp, memana, mearmor, memanaspent, bosshp, bossdmg, effects):
    #simulates a boss turn
    #returns the minimum mana spent to win for player
    #apply effects
    if effects[0] > 0:
        bosshp -= 4
        effects[0] -= 1
    if effects[1] > 0:
        mearmor = 7
        effects[1] -= 1
    if effects[2] > 0:
        mehp += 101
        effects[2] -= 1
    if effects[3] > 0:
        effects[3] -= 1
    if effects[4] > 0:
        memana += 101
        effects[4] -= 1
    #check if boss is dead
    if bosshp <= 0:
        return (True, memanaspent)
    #check if player is dead
    if mehp <= 0:
        return (False, memanaspent)
    #simulate boss turn
    newHp = mehp - max(1, bossdmg - mearmor)
    newMana = memana
    newArmor = mearmor
    newManaSpent = memanaspent
    newEffects = effects.copy()
    return simulatePlayer(newHp, newMana, newArmor, newManaSpent, bosshp, bossdmg, newEffects)



In [32]:
print(simulatePlayer(meHp, meMana, meArmor, meManaSpent, bossHp, bossDmg, (0,0,0,0,0)))

TypeError: 'tuple' object does not support item assignment